In [ ]:
!python -m pip install pycld2

In [ ]:
raw_corpus = {
    "hu": "../tokenizers/wikipedia_hu_all_nopic_2024-05.text",
    "et": "../tokenizers/wikipedia_et_all_nopic_2024-05.text",
    "fi": "../tokenizers/wikipedia_fi_all_nopic_2024-05.text",
    "se": "../tokenizers/wikipedia_se_all_nopic_2024-05.text",
}
step1_corpus = {
    "hu": "../tokenizers/wikipedia_hu_all_nopic_2024-05.step1.text",
    "et": "../tokenizers/wikipedia_et_all_nopic_2024-05.step1.text",
    "fi": "../tokenizers/wikipedia_fi_all_nopic_2024-05.step1.text",
    "se": "../tokenizers/wikipedia_se_all_nopic_2024-05.step1.text",
}

In [ ]:
### step 1. Lets remove all not-latin characters
import unicodedata
import re
from tqdm import tqdm

def is_latin(word):
    return all(unicodedata.name(char).startswith('LATIN') for char in word)

def filter_latin_words(text):
    words = re.findall(r'\b\w+\b', text)
    return [word for word in words if is_latin(word)]

for language, corpus_file in raw_corpus.items():
    print(language)
    with open(corpus_file, 'r') as f:
        with open(step1_corpus[language], 'w') as out:
            for line in f:
                filtered_words = filter_latin_words(line)
                if len(filtered_words) < 10:
                    continue
                out.write(' '.join(filtered_words) + '\n')

In [ ]:
uralic_cons_or_not = {
    "HUNGARIAN": 0,
    "HUNGARIAN not": 0,
    "FINNISH": 0,
    "FINNISH not": 0,
    "ESTONIAN": 0,
    "ESTONIAN not": 0,
    "NORTHERN SAMI": 0,
    "NORTHERN SAMI not": 0
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/MyDrive/uralic/uralic.txt', 'r', encoding='utf-8') as file:
    articles = file.read().split('\n')

In [ ]:
### Step 2. Lets recheck the languages wtih pycld2

import pycld2 as cld2
from tqdm.auto import tqdm

file_name = "monolingual.txt"

with open(file_name, "w") as file:
    for i, article in enumerate(tqdm(articles), start=1):
        if article.strip():
            try:
                is_reliable, text_bytes_found, details = cld2.detect(article)
            except:
                continue

            if is_reliable:
                lang_name = details[0][0]
                lang_percentage = details[0][2]
                if lang_name in ["HUNGARIAN", "FINNISH", "ESTONIAN", "NORTHERN SAMI"]:
                    # if lang_percentage > 94:
                    if details[1][0] == "Unknown":
                        uralic_cons_or_not[lang_name] += 1
                        file.write(article + "\n")
                    else:
                        uralic_cons_or_not[f"{lang_name} not"] += 1
            else:
                continue

In [ ]:
uralic_cons_or_not

In [ ]:
from google.colab import files

files.download("monolingual.txt")